In [47]:
import pandas as pd

# Importing the dataset
deposit_df = pd.read_csv("Deposit Data.csv")
gameplay_df = pd.read_csv("User Gameplay data.csv")
withdrawal_df = pd.read_csv("Withdrawal Data.csv")

# Displaying the first five rows
deposit_df.head(), gameplay_df.head(), withdrawal_df.head()


(   User Id         Datetime  Amount
 0      357  01-10-2022 0:03    2000
 1      776  01-10-2022 0:03    2500
 2      492  01-10-2022 0:06    5000
 3      803  01-10-2022 0:07    5000
 4      875  01-10-2022 0:09    1500,
    User ID  Games Played         Datetime
 0      851             1  01-10-2022 0:00
 1      717             1  01-10-2022 0:00
 2      456             1  01-10-2022 0:00
 3      424             1  01-10-2022 0:00
 4      845             1  01-10-2022 0:00,
    User Id         Datetime  Amount
 0      190  01-10-2022 0:03    5872
 1      159  01-10-2022 0:16    9540
 2      164  01-10-2022 0:24     815
 3      946  01-10-2022 0:29   23000
 4      763  01-10-2022 0:40    9473)

In [48]:
slot_points = gameplay_df.copy()

# Proper datetime format
slot_points['Datetime'] = pd.to_datetime(slot_points['Datetime'], errors='coerce')
slot_points = slot_points.dropna(subset=['Datetime']) 

# Adding Slot and Date
slot_points['Slot'] = slot_points['Datetime'].apply(lambda dt: 'S1' if dt.hour < 12 else 'S2')
slot_points['Date'] = slot_points['Datetime'].dt.date

# Calculate Loyalty Points
slot_points['Loyalty Points'] = slot_points['Games Played'] * 10

# Filter for target dates and slots
target_dates = [('2022-10-02', 'S1'), ('2022-10-16', 'S2'), ('2022-10-18', 'S1'), ('2022-10-26', 'S2')]
target_dates = [(pd.to_datetime(d).date(), s) for d, s in target_dates]

filtered = slot_points[
    slot_points[['Date', 'Slot']].apply(tuple, axis=1).isin(target_dates)
]

# Group by User ID
result = filtered.groupby(['User ID', 'Date', 'Slot'])['Loyalty Points'].sum().reset_index()
print(result.sort_values(by=['Date', 'Slot', 'Loyalty Points'], ascending=[True, True, False]))


Empty DataFrame
Columns: [User ID, Date, Slot, Loyalty Points]
Index: []


In [ ]:
# datetime
gameplay_df['Datetime'] = pd.to_datetime(gameplay_df['Datetime'], errors='coerce')
gameplay_df = gameplay_df.dropna(subset=['Datetime'])

# Filter October data
october_data = gameplay_df[gameplay_df['Datetime'].dt.month == 10]

# Group and calculate
ranking = october_data.groupby('User ID').agg(
    Total_Games=('Games Played', 'sum')
).assign(
    Loyalty_Points=lambda x: x['Total_Games'] * 10
).sort_values(by=['Loyalty_Points', 'Total_Games'], ascending=False).reset_index()

# View top 5
ranking.head()


,User ID,Total_Games,Loyalty_Points
0,765,771,7710
1,618,224,2240
2,247,214,2140
3,663,205,2050
4,449,197,1970


In [28]:
# Average Deposit Amount
avg_deposit = deposit_df['Amount'].mean()
print(avg_deposit)

5492.185399701801


In [30]:

monthly_user_deposit = deposit_df.groupby('User Id')['Amount'].sum()
avg_per_user_deposit = monthly_user_deposit.mean()
print(monthly_user_deposit)
print(avg_per_user_deposit)


User Id
1        5000
2      567000
3       40000
4        1750
5       74100
        ...  
994      1400
995     34500
996     96400
998      7400
999      9000
Name: Amount, Length: 915, dtype: int64
104669.64918032786


In [31]:
avg_games_per_user = gameplay_df.groupby('User ID')['Games Played'].sum().mean()
print(avg_games_per_user)

145.91198303287382


### Part B - Bonus to TOP 50 players

We assign a bonus pot of 50,000INR to the first 50 players according to a weighted score of the loyalty points and the number of played games.

**Formula:**

Player Score = 0.7 (Loyalty Points / Max Loyalty Points) + 0.3 (Games Played/ Max Games Played )

Bonus = (Player Score / Sum of all 50 scores )(50000).


In [46]:
# Use best 50 candidates in earlier ranking DataFrame
top_50 = ranking.head(50).copy()

# Normalize values
top_50['Norm_Loyalty'] = top_50['Loyalty_Points'] / top_50['Loyalty_Points'].max()
top_50['Norm_Games'] = top_50['Total_Games'] / top_50['Total_Games'].max()

# Weighted score
top_50['Score'] = 0.7 * top_50['Norm_Loyalty'] + 0.3 * top_50['Norm_Games']

# Distribution of bonus goodies
total_score = top_50['Score'].sum()
top_50['Bonus'] = (top_50['Score'] / total_score) * 50000

# Final output
bonus_distribution = top_50[['User ID', 'Loyalty_Points', 'Total_Games', 'Bonus']].sort_values(by='Bonus', ascending=False)
print(bonus_distribution)


    User ID  Loyalty_Points  Total_Games        Bonus
0       765            7710          771  6271.351879
1       618            2240          224  1822.027005
2       247            2140          214  1740.686514
3       663            2050          205  1667.480072
4       449            1970          197  1602.407679
5       856            1940          194  1578.005531
6       754            1850          185  1504.799089
7       887            1760          176  1431.592647
8       456            1730          173  1407.190499
9       502            1690          169  1374.654303
10      738            1690          169  1374.654303
11       39            1430          143  1163.169026
12      365            1350          135  1098.096633
13        9            1310          131  1065.560436
14      990            1290          129  1049.292338
15      989            1260          126  1024.890190
16      137            1230          123  1000.488043
17      722            1230 

## Part C Fairness of the Loyalty Point Formula

❌ The Current Formula:
Player Score = 0.7 (Loyalty Points / Max Loyalty Points) + 0.3 (Games Played/ Max Games Played )

Bonus = (Player Score / Sum of all 50 scores )(50000).

# Problems of the existing formula:
- **Treatment of all games is equal** - No distinction is made between games on the basis of type, duration or value.

- **No reason to get skilled or to win** - players will get points no matter if they win or lose.

- **Promotes spamming** - more rewarded by the quantity rather than quality or involvement.

- **Does not include deposit or transaction attitude** - a trusting financial contributor father is not rewarded.


# Some Recommendations on How to Make It Stronger:

Dimension And Recommendation of improvement

Game Outcome Reward exceeds loss (e.g. 10point more than 5 if win)/5 and vice versa (Loss).

**Stake Size** - Weigh each loyalty point on the basis of amount of bet or value of game.

**Consistency** - Put streak bonuses on consecutive active days (e.g., 5 days streak and more = bonus).

**Daily Limits** - Reduce the max point limit on a daily basis so as to discourage bots/spam based loyalty point farming.

**Deposit Link** - Add deposit action (e.g. 1 loyalty point every 100 ₹ deposited).

**Diversity** - Promote play with a variety of games by making use of added variants generators.

proposes a possible alternate formula.



***In Example 1:*** Balanced Behavior-Based Points, 11 of 12 weighted points were awarded (or 92 percent of total weighted points were awarded or delivered).

Formula Can be: Loyalty Points = (Games Played × 8) + (Games Won × 5) + (Active Days × 3)

***In Example 2:*** Stake-Aware Loyalty Points

Formula Can be: Loyalty Points = Games Played × log(1 + Total Bet Amount)

### 🎯 In Conclusion:
The present model is **closeretic and easy**. A more subtle formula has the merit:
- **Fairness** (rewarding and engagement that is real)
- **Retention** (the driver of users playing on)
- **Deterrence of abuse** (by deterring shoddy or fake gaming behavior)